In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import pandas as pd
import json
from os import path
from matplotlib import pyplot as plt

In [ ]:
images_dir = '../input/tensorflow-great-barrier-reef/train_images'
train_csv_path = '../input/tensorflow-great-barrier-reef/train.csv'

In [ ]:
train_df = pd.read_csv(train_csv_path)
train_df['annotations'] = train_df['annotations'].str.replace("'", '"')
train_df['annotations'] = train_df['annotations'].apply(json.loads)
train_df['num_fishes'] = train_df['annotations'].apply(len)
train_df_with_fishes = train_df[train_df['num_fishes']>0].reset_index(drop=True)

In [ ]:
train_df['num_fishes'].hist()

In [ ]:
def get_image(img_dir, video_id, video_frame):
    img_sub_path = "video_{}/{}.jpg".format(video_id, video_frame)
    img_path = path.join(img_dir, img_sub_path)
    return Image.open(img_path)
    
def draw_bounding_boxes(img, annotations):
    for box in annotations:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="red")
    return img

def get_random_image(df):
    id = np.random.randint(df.shape[0])
    img_row = df.iloc[id]
    img = get_image(images_dir, img_row['video_id'], img_row['video_frame'])
    return img, img_row

def showImage(df):
    img, img_row = get_random_image(df)
    display(draw_bounding_boxes(img, img_row['annotations'])) 

def showImageDiff(df):
    img, img_row = get_random_image(df)
    prev_img = get_image(images_dir, img_row['video_id'], img_row['video_frame']-1)
    diff = ImageChops.difference(prev_img, img)
  
    if diff.getbbox():
        dsplay(draw_bounding_boxes(diff, img_row['annotations']))

def plotImageHistogram(df):
    def plotImageHist(img_arr):
        plt.hist(img_arr.ravel(), bins=256)
        plt.show()
    img, img_row = get_random_image(df)
    img_arr = np.array(img)
    display(draw_bounding_boxes(img, img_row['annotations']))
    plotImageHist(img_arr)
    for box in img_row['annotations']:
        sub_img_arr = img_arr[box['y']:box['y']+box['height'], box['x']:box['x']+box['width']]
        plotImageHist(sub_img_arr)

In [ ]:
showImage(train_df_with_fishes)

In [ ]:
plotImageHistogram(train_df_with_fishes)